In [88]:
import requests
import json
import time
import os

from pprint import pprint
import pandas as pd
pd.set_option('display.precision', 2)

access_token = 'eea5921ffd90fa3c6301ce3970a8bc6ed35ef34b'

from IPython.core.display import HTML
css = open('style-table.css').read() + open('style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

import sys
sys.executable

'/Users/edwardterrell/opt/anaconda3/envs/scrape311/bin/python'

In [89]:
pwd

'/Users/edwardterrell/Desktop/Training/kindle/strava'

In [90]:
from token_manager import get_valid_access_token
import requests

access_token = get_valid_access_token()
print("Using access token:", access_token)

# Make an authenticated API call
response = requests.get(
    'https://www.strava.com/api/v3/athlete',
    headers={'Authorization': f'Bearer {access_token}'}
)

print(response.json())

Using access token: 71a22caf16418f0068141e8413120320b3fde178
{'id': 723727, 'username': 'eterrell', 'resource_state': 2, 'firstname': 'Ed', 'lastname': 'Terrell', 'bio': '', 'city': 'Boulder', 'state': 'CO', 'country': 'United States', 'sex': 'M', 'premium': True, 'summit': True, 'created_at': '2012-07-05T00:04:09Z', 'updated_at': '2025-06-19T23:51:05Z', 'badge_type_id': 1, 'weight': 61.235, 'profile_medium': 'https://dgalywyr863hv.cloudfront.net/pictures/athletes/723727/20826329/1/medium.jpg', 'profile': 'https://dgalywyr863hv.cloudfront.net/pictures/athletes/723727/20826329/1/large.jpg', 'friend': None, 'follower': None}


In [ ]:
# USED ONLY IF token_manager.py isn't generating correctly
client_id = 167221
client_secret = 'f4df8e0dec7d9d8e725707eaf05037fcc845989d'
auth_code = 'd973f0f86d2e56c3e7ac369809ca6b14fc814a98' # stored from last time

response = requests.post(
    'https://www.strava.com/oauth/token',
    data={
        'client_id': client_id,
        'client_secret': client_secret,
        'code': auth_code,
        'grant_type': 'authorization_code',
    }
)
# Check and save response
if response.status_code == 200:
    tokens = response.json()
    print("✅ Tokens received:")
    print(json.dumps(tokens, indent=2))

    # Save to tokens.json for use in token_manager.py
    with open("tokens.json", "w") as f:
        json.dump(tokens, f)
else:
    print("❌ Failed to get tokens:", response.status_code)
    print(response.text)

In [91]:
access_token

'71a22caf16418f0068141e8413120320b3fde178'

In [92]:
import datetime
expires_at = 1751931129
# Convert to datetime object (local time)
dt_local = datetime.datetime.fromtimestamp(expires_at)
print("Local time: ", dt_local)

Local time:  2025-07-07 17:32:09


In [93]:
# Get your recent activities
response = requests.get(
    'https://www.strava.com/api/v3/athlete/activities',
    headers={'Authorization': f'Bearer {access_token}'}
)
activities = response.json()
response.status_code

200

In [94]:
# Show the first 3 activities with ID and name
for activity in activities[:3]:
    print(f"ID: {activity['id']}, Name: {activity['name']}, Date: {activity['start_date']}")

ID: 15037283785, Name: Old Stage, Date: 2025-07-07T12:38:38Z
ID: 15027982989, Name: North dirt roads to Longmont bike paths and Babettes, Date: 2025-07-06T14:14:42Z
ID: 15017668877, Name: Base of LickSkillet++, Date: 2025-07-05T13:55:37Z


In [95]:
# activity ID you want to inspect
activity_id = 15006491752

# Define endpoint URL
url = f'https://www.strava.com/api/v3/activities/{activity_id}/zones'
# Set up Authorization header and make request
headers = {'Authorization': f'Bearer {access_token}'}
response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    zones = response.json()
    pprint(zones[0])
else:
    print(f"Error: {response.status_code}")
    print(response.text)

{'custom_zones': False,
 'distribution_buckets': [{'max': 93, 'min': 0, 'time': 63.0},
                          {'max': 123, 'min': 93, 'time': 4472.0},
                          {'max': 138, 'min': 123, 'time': 4347.0},
                          {'max': 153, 'min': 138, 'time': 1498.0},
                          {'max': -1, 'min': 153, 'time': 0.0}],
 'points': 33,
 'resource_state': 3,
 'score': 230.0,
 'sensor_based': True,
 'type': 'heartrate'}


In [96]:
zone_series  = pd.DataFrame(zones[0]).iloc[:,1]
zone_series

0         {'min': 0, 'max': 93, 'time': 63.0}
1     {'min': 93, 'max': 123, 'time': 4472.0}
2    {'min': 123, 'max': 138, 'time': 4347.0}
3    {'min': 138, 'max': 153, 'time': 1498.0}
4        {'min': 153, 'max': -1, 'time': 0.0}
Name: distribution_buckets, dtype: object

In [98]:
suffer_score = pd.DataFrame(zones[0]).iloc[0,0]
print(suffer_score)
time_in_zones = zone_series.apply(lambda z: z['time']/60)
time_in_zones

230.0


0     1.05
1    74.53
2    72.45
3    24.97
4     0.00
Name: distribution_buckets, dtype: float64

In [99]:
for activity in activities:
    print(f"ID: {activity['id']}, Type: {activity['sport_type']}, Name: {activity['name']}, Date: {activity['start_date']}")

ID: 15037283785, Type: Ride, Name: Old Stage, Date: 2025-07-07T12:38:38Z
ID: 15027982989, Type: Ride, Name: North dirt roads to Longmont bike paths and Babettes, Date: 2025-07-06T14:14:42Z
ID: 15017668877, Type: Ride, Name: Base of LickSkillet++, Date: 2025-07-05T13:55:37Z
ID: 15006491752, Type: Ride, Name: Jamestown - Tempo, Date: 2025-07-04T13:46:37Z
ID: 14975645989, Type: Ride, Name: Sunshine Canyon and Wallstreet to base of Switzerland Trail, Date: 2025-07-01T13:58:30Z
ID: 14964904782, Type: Ride, Name: Gravel roads north, Date: 2025-06-30T14:38:54Z
ID: 14954454448, Type: Ride, Name: Olde Stage, Date: 2025-06-29T14:11:46Z
ID: 14944687171, Type: Ride, Name: Ned gravel loop, Date: 2025-06-28T16:10:10Z
ID: 14934728422, Type: Ride, Name: St. Louis  Creek  to WP ski area, Date: 2025-06-27T15:23:35Z
ID: 14925456666, Type: Ride, Name: RCMP VC Trail Ridge East  Side, Date: 2025-06-26T15:01:06Z
ID: 14915319034, Type: Ride, Name: Trail Ridge into the hailstorm, Date: 2025-06-25T14:39:32Z
ID:

In [100]:
strava_df = pd.DataFrame(activities)
cols_needed = ['id','name', 'distance', 'moving_time','total_elevation_gain', 'sport_type',
          'average_speed', 'average_heartrate','max_heartrate', 'suffer_score','start_date']
strava_df = strava_df.loc[:,cols_needed]
strava_df.head(3)

,id,name,distance,moving_time,total_elevation_gain,sport_type,average_speed,average_heartrate,max_heartrate,suffer_score,start_date
0,15037283785,Old Stage,44640.8,7128,430.0,Ride,6.26,109.6,134.0,54.0,2025-07-07T12:38:38Z
1,15027982989,North dirt roads to Longmont bike paths and Ba...,59511.9,10021,361.6,Ride,5.94,102.5,129.0,47.0,2025-07-06T14:14:42Z
2,15017668877,Base of LickSkillet++,75306.7,13372,1087.6,Ride,5.63,109.9,141.0,117.0,2025-07-05T13:55:37Z


In [101]:
# Change to datetime and creat new date column
strava_df.start_date = pd.to_datetime(strava_df.start_date) 
strava_df['date'] = strava_df.start_date.dt.date

# Change distance to miles
strava_df['distance'] = strava_df['distance'] / 1609.34

# Change moving time to minutes in new col: time
strava_df['time'] = strava_df.moving_time/60

# Convert to timedelta
strava_df.moving_time = pd.to_timedelta(strava_df['moving_time'], unit='s')

# Change to hours/minutes
strava_df['moving_time'] = strava_df['moving_time'].apply(lambda td: f"{int(td.total_seconds() // 3600)}:{int((td.total_seconds() % 3600) // 60):02}")

# Change to feet
strava_df.total_elevation_gain = strava_df.total_elevation_gain*3.28084

In [102]:
def convert_speed(row):
    if row['sport_type'] == 'Ride':
        return row['average_speed'] * 2.23694  # m/s to mph
    elif row['sport_type'] == 'Run':
        if row['average_speed'] > 0:
            pace_sec_per_mile = 1609.34 / row['average_speed']
            minutes = int(pace_sec_per_mile // 60)
            seconds = int(pace_sec_per_mile % 60)
            return f"{minutes}:{seconds:02d}"
        else:
            return "N/A"
    else:
        return None  # for other sports

strava_df['converted_speed'] = strava_df.apply(convert_speed, axis=1)


In [103]:
new_order = ['id', 'name','date','sport_type',  'distance','moving_time', 'converted_speed', 
       'total_elevation_gain', 'average_heartrate', 'max_heartrate',
       'suffer_score', 'time','average_speed','start_date' ]

In [104]:
pd.set_option('display.precision', 1)
strava_df = strava_df[new_order]
strava_df.head(2)

,id,name,date,sport_type,distance,moving_time,converted_speed,total_elevation_gain,average_heartrate,max_heartrate,suffer_score,time,average_speed,start_date
0,15037283785,Old Stage,2025-07-07,Ride,27.7,1:58,14.0,1410.8,109.6,134.0,54.0,118.8,6.3,2025-07-07 12:38:38+00:00
1,15027982989,North dirt roads to Longmont bike paths and Ba...,2025-07-06,Ride,37.0,2:47,13.3,1186.4,102.5,129.0,47.0,167.0,5.9,2025-07-06 14:14:42+00:00


In [105]:
strava_df.columns = ['id', 'name', 'date', 'sport', 'distance', 'time',
       'speed', 'elev_gain',
       'avg_HR', 'max_HR', 'suffer_score', 'time-minutes','average_speed',
       'start_date']

In [106]:
strava_df.head(4)

,id,name,date,sport,distance,time,speed,elev_gain,avg_HR,max_HR,suffer_score,time-minutes,average_speed,start_date
0,15037283785,Old Stage,2025-07-07,Ride,27.7,1:58,14.0,1410.8,109.6,134.0,54.0,118.8,6.3,2025-07-07 12:38:38+00:00
1,15027982989,North dirt roads to Longmont bike paths and Ba...,2025-07-06,Ride,37.0,2:47,13.3,1186.4,102.5,129.0,47.0,167.0,5.9,2025-07-06 14:14:42+00:00
2,15017668877,Base of LickSkillet++,2025-07-05,Ride,46.8,3:42,12.6,3568.2,109.9,141.0,117.0,222.9,5.6,2025-07-05 13:55:37+00:00
3,15006491752,Jamestown - Tempo,2025-07-04,Ride,39.1,2:53,13.5,2710.0,125.1,147.0,230.0,173.0,6.1,2025-07-04 13:46:37+00:00


In [107]:
# activity ID to inspect
activity_id = 15006491752
# endpoint URL
url = f'https://www.strava.com/api/v3/activities/{activity_id}/zones'
# Set up Authorization header and make request
headers = {'Authorization': f'Bearer {access_token}'}
response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    zones = response.json()
else:
    print(f"Error: {response.status_code}")
    print(response.text)

In [108]:
activity_df = strava_df.loc[strava_df.id == 15006491752, ['name','date']]

In [109]:
activity_df

,name,date
3,Jamestown - Tempo,2025-07-04


In [115]:
Date = activity_df.iloc[0,1]
print(f"Date: ", Date)
name = activity_df.iloc[0,0]
print(f"Name: ", name)

suffer_score = pd.DataFrame(zones[0]).iloc[0,0]
print(f"Suffer Score: ",suffer_score)
zone_series  = pd.DataFrame(zones[0]).iloc[:,1]
time_in_zones = zone_series.apply(lambda z: z['time']/60)
total_minutes = time_in_zones.sum()
print (f"Total Time: {total_minutes:.1f} minutes")
time_in_zones

Date:  2025-07-04
Name:  Jamestown - Tempo
Suffer Score:  230.0
Total Time: 173.0 minutes


0     1.1
1    74.5
2    72.5
3    25.0
4     0.0
Name: distribution_buckets, dtype: float64

In [ ]:
activity_df['suffer_score'] = pd.DataFrame(zones[0]).iloc[0,0]
activity_df['total_minutes'] = time_in_zones.sum()

In [120]:
activity_df

,name,date,suffer_score,total_minutes
3,Jamestown - Tempo,2025-07-04,230.0,173.0


In [121]:
zone_series  = pd.DataFrame(zones[0]).iloc[:,1]
time_in_zones = zone_series.apply(lambda z: z['time']/60)
time_in_zones

0     1.1
1    74.5
2    72.5
3    25.0
4     0.0
Name: distribution_buckets, dtype: float64

In [122]:
time_in_zones.to_frame('times')

,times
0,1.1
1,74.5
2,72.5
3,25.0
4,0.0


In [123]:
# Create new column names
column_names = [f'Zone{i+1}' for i in range(len(time_in_zones))]

# Convert to single-row DataFrame
zone_df = pd.DataFrame([time_in_zones.values], columns=column_names)

In [125]:
zone_df.index = activity_df.index
zone_df

,Zone1,Zone2,Zone3,Zone4,Zone5
3,1.1,74.5,72.5,25.0,0.0


In [129]:
activity_df = pd.merge(activity_df, zone_df,left_index=True, right_index=True)
activity_df

,name,date,suffer_score,total_minutes,Zone1,Zone2,Zone3,Zone4,Zone5
3,Jamestown - Tempo,2025-07-04,230.0,173.0,1.1,74.5,72.5,25.0,0.0


In [130]:
activity_df['moderate'] = activity_df.Zone1 + activity_df.Zone2
activity_df['hard']= activity_df.Zone3 + activity_df.Zone4 + activity_df.Zone5

In [131]:
activity_df

,name,date,suffer_score,total_minutes,Zone1,Zone2,Zone3,Zone4,Zone5,moderate,hard
3,Jamestown - Tempo,2025-07-04,230.0,173.0,1.1,74.5,72.5,25.0,0.0,75.6,97.4


### Listing what is available 

In [ ]:
from pprint import pprint
n=0
for activity in activities:
    n+=1
    if n <= 1:
        pprint(activity)